In [4]:
# Install the yfinance library using pip
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip list

Package                           Version
--------------------------------- ------------------
aext-assistant                    4.1.0
aext-assistant-server             4.1.0
aext-core                         4.1.0
aext-core-server                  4.1.0
aext-panels                       4.1.0
aext-panels-server                4.1.0
aext-project-filebrowser-server   4.1.0
aext-share-notebook               4.1.0
aext-share-notebook-server        4.1.0
aext-shared                       4.1.0
aext-toolbox                      4.1.0
aiobotocore                       2.12.3
aiohappyeyeballs                  2.4.0
aiohttp                           3.10.5
aioitertools                      0.7.1
aiosignal                         1.2.0
alabaster                         0.7.16
alembic                           1.13.3
altair                            5.0.1
anaconda-anon-usage               0.4.4
anaconda-catalogs                 0.2.0
anaconda-cli-base                 0.4.1
anaconda-client      

In [1]:
# Load libraries
import yfinance as yf
import pandas as pd
from datetime import datetime

In [3]:
# Get S&P 500 tickers from yahoo finance
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
sp500_table = pd.read_html(url)[0]
sp500_table["Symbol"] = sp500_table["Symbol"].str.strip().str.upper()
sp500_table["Security"] = sp500_table["Security"].str.strip()
ticker_name_map = dict(zip(sp500_table["Symbol"], sp500_table["Security"]))
tickers = list(ticker_name_map.keys())
print(tickers[:10])

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [9]:
# Load dividend aristocrats from local CSV
aristocrats_df = pd.read_excel("aristocrats.csv.xlsx")
aristocrat_tickers = aristocrats_df["Ticker"].str.strip().str.upper().tolist()
aristocrat_tickers

['BDX',
 'PEP',
 'NDSN',
 'PPG',
 'TGT',
 'SYY',
 'NUE',
 'HRL',
 'LOW',
 'JNJ',
 'SPGI',
 'GPC',
 'MDT',
 'EMR',
 'BF.B',
 'ALB',
 'FDS',
 'ADM',
 'ITW',
 'GWW',
 'MKC',
 'KO',
 'AOS',
 'ADP',
 'ROP',
 'CHD',
 'APD',
 'BRO',
 'DOV',
 'CL',
 'ABBV',
 'CAT',
 'ATO',
 'PNR',
 'GD',
 'ABT',
 'MCD',
 'ERIE',
 'PG',
 'CB',
 'CTAS',
 'WMT',
 'SHW',
 'AFL',
 'WST',
 'LIN',
 'CINF',
 'ECL',
 'KVUE',
 'ES',
 'TROW',
 'SWK',
 'NEE',
 'CLX',
 'BEN',
 'ED',
 'KMB',
 'FRT',
 'EXPD',
 'CAH',
 'SJM',
 'ESS',
 'CHRW',
 'FAST',
 'O',
 'XOM',
 'IBM',
 'AMCR',
 'CVX']

In [15]:
# Define the period of analysis
start_date = "2025-01-01"
end_date = "2025-06-30"

In [17]:
# Get stock data from yahoo finance
def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        
        hist = stock.history(start=start_date, end=end_date)
        if hist.empty:
            print(f"{ticker}: No price data - retrying once...")
            time.sleep(0.5)
            hist = stock.history(start=start_date, end=end_date)

        if hist.empty:
            print(f"{ticker}: Still no price data - skipping.")
            return None

        # Calculate price return + momentum
        start_price = hist.iloc[0]["Close"]
        end_price = hist.iloc[-1]["Close"]
        price_return = ((end_price - start_price) / start_price) * 100
        momentum = end_price - start_price

        # Volatility (Daily returns standard deviation)
        hist["Daily Return"] = hist["Close"].pct_change()
        volatility = hist["Daily Return"].std() * (252 ** 0.5) * 100
       
        # Dividend Yield
        dividends = stock.dividends
        q1_dividends = dividends.loc[(dividends.index >= start_date) & (dividends.index <= end_date)].sum()
        dividend_yield = (q1_dividends / start_price) * 100

        # Liquidity
        avg_volume = hist["Volume"].mean()

        # Total Return
        total_return = price_return + dividend_yield

        # Extract fundamentals from yahoo finance
        try:
            info = stock.info
            roe = info.get("returnOnEquity", None)
            roe = roe * 100 if roe is not None else None
            pe_ratio = info.get("trailingPE", None)
        except:
            roe = None
            pe_ratio = None
        
        return {
        "Ticker": ticker,
        "Company Name": ticker_name_map.get(ticker, ""),
        "Start Price": round(start_price, 2),
        "End Price": round(end_price, 2),
        "Price Return %": round(price_return, 2),
        "Q1 Dividends ($)": round(q1_dividends, 2),
        "Dividend Yield %": round(dividend_yield, 2),
        "Total Return %": round(total_return, 2),
        "Volatility %": round(volatility, 2),
        "Avg Daily Volume": round(avg_volume),
        "Momentum ($)": round(momentum, 2),
        "ROE": round(roe, 2) if roe is not None else None,
        "P/E Ratio": round(pe_ratio, 2) if pe_ratio is not None else None
        
        }
    except Exception as e:
        return None

In [19]:
# Loop through over 500 S&P tickers and fetch stock metrics
results = []

for i, ticker in enumerate(tickers):
    ticker = ticker.replace(".", "-")
    print(f"[{i+1}/{len(tickers)}] Checking: {ticker}")
    data = get_stock_data(ticker)
    if data:
        print(" Appending:", data)
        results.append(data)

print(f"\n✅ Completed. Total stocks found: {len(results)}")

[1/502] Checking: MMM
 Appending: {'Ticker': 'MMM', 'Company Name': '3M', 'Start Price': 128.43, 'End Price': 152.02, 'Price Return %': 18.36, 'Q1 Dividends ($)': 1.46, 'Dividend Yield %': 1.14, 'Total Return %': 19.5, 'Volatility %': 34.98, 'Avg Daily Volume': 3878507, 'Momentum ($)': 23.59, 'ROE': 93.82, 'P/E Ratio': 19.05}
[2/502] Checking: AOS
 Appending: {'Ticker': 'AOS', 'Company Name': 'A. O. Smith', 'Start Price': 66.65, 'End Price': 65.65, 'Price Return %': -1.5, 'Q1 Dividends ($)': 0.68, 'Dividend Yield %': 1.02, 'Total Return %': -0.48, 'Volatility %': 26.8, 'Avg Daily Volume': 1582531, 'Momentum ($)': -1.0, 'ROE': 27.97, 'P/E Ratio': 18.97}
[3/502] Checking: ABT
 Appending: {'Ticker': 'ABT', 'Company Name': 'Abbott Laboratories', 'Start Price': 112.33, 'End Price': 134.38, 'Price Return %': 19.63, 'Q1 Dividends ($)': 1.18, 'Dividend Yield %': 1.05, 'Total Return %': 20.68, 'Volatility %': 23.48, 'Avg Daily Volume': 6853778, 'Momentum ($)': 22.05, 'ROE': 30.65, 'P/E Ratio': 

$EMN: possibly delisted; no price data found  (1d 1926-07-29 -> 2025-07-04)


[160/502] Checking: ETN
 Appending: {'Ticker': 'ETN', 'Company Name': 'Eaton Corporation', 'Start Price': 329.6, 'End Price': 353.23, 'Price Return %': 7.17, 'Q1 Dividends ($)': 2.08, 'Dividend Yield %': 0.63, 'Total Return %': 7.8, 'Volatility %': 45.31, 'Avg Daily Volume': 3237667, 'Momentum ($)': 23.63, 'ROE': 20.81, 'P/E Ratio': 36.51}
[161/502] Checking: EBAY
 Appending: {'Ticker': 'EBAY', 'Company Name': 'eBay Inc.', 'Start Price': 61.74, 'End Price': 73.76, 'Price Return %': 19.47, 'Q1 Dividends ($)': 0.58, 'Dividend Yield %': 0.94, 'Total Return %': 20.4, 'Volatility %': 32.68, 'Avg Daily Volume': 5589391, 'Momentum ($)': 12.02, 'ROE': 36.48, 'P/E Ratio': 18.36}
[162/502] Checking: ECL
 Appending: {'Ticker': 'ECL', 'Company Name': 'Ecolab', 'Start Price': 229.77, 'End Price': 267.45, 'Price Return %': 16.4, 'Q1 Dividends ($)': 1.3, 'Dividend Yield %': 0.57, 'Total Return %': 16.96, 'Volatility %': 22.56, 'Avg Daily Volume': 1366707, 'Momentum ($)': 37.68, 'ROE': 24.77, 'P/E Rat

$GPC: possibly delisted; no price data found  (1d 1926-07-29 -> 2025-07-04)


[217/502] Checking: GILD
 Appending: {'Ticker': 'GILD', 'Company Name': 'Gilead Sciences', 'Start Price': 90.6, 'End Price': 110.67, 'Price Return %': 22.16, 'Q1 Dividends ($)': 1.58, 'Dividend Yield %': 1.74, 'Total Return %': 23.9, 'Volatility %': 30.1, 'Avg Daily Volume': 8984555, 'Momentum ($)': 20.07, 'ROE': 32.65, 'P/E Ratio': 23.48}
[218/502] Checking: GPN
 Appending: {'Ticker': 'GPN', 'Company Name': 'Global Payments', 'Start Price': 111.24, 'End Price': 78.72, 'Price Return %': -29.23, 'Q1 Dividends ($)': 0.5, 'Dividend Yield %': 0.45, 'Total Return %': -28.78, 'Volatility %': 47.21, 'Avg Daily Volume': 2930748, 'Momentum ($)': -32.52, 'ROE': 7.1, 'P/E Ratio': 13.24}
[219/502] Checking: GL
 Appending: {'Ticker': 'GL', 'Company Name': 'Globe Life', 'Start Price': 111.52, 'End Price': 123.61, 'Price Return %': 10.84, 'Q1 Dividends ($)': 0.51, 'Dividend Yield %': 0.46, 'Total Return %': 11.3, 'Volatility %': 28.57, 'Avg Daily Volume': 634956, 'Momentum ($)': 12.09, 'ROE': 20.47, 

$MTB: possibly delisted; no price data found  (1d 1926-07-29 -> 2025-07-04)


[298/502] Checking: MPC
 Appending: {'Ticker': 'MPC', 'Company Name': 'Marathon Petroleum', 'Start Price': 139.89, 'End Price': 167.41, 'Price Return %': 19.67, 'Q1 Dividends ($)': 1.82, 'Dividend Yield %': 1.3, 'Total Return %': 20.98, 'Volatility %': 41.49, 'Avg Daily Volume': 2851983, 'Momentum ($)': 27.52, 'ROE': 15.52, 'P/E Ratio': 24.28}
[299/502] Checking: MKTX
 Appending: {'Ticker': 'MKTX', 'Company Name': 'MarketAxess', 'Start Price': 223.65, 'End Price': 219.55, 'Price Return %': -1.83, 'Q1 Dividends ($)': 1.52, 'Dividend Yield %': 0.68, 'Total Return %': -1.15, 'Volatility %': 27.71, 'Avg Daily Volume': 620330, 'Momentum ($)': -4.1, 'ROE': 16.25, 'P/E Ratio': 37.56}
[300/502] Checking: MAR
 Appending: {'Ticker': 'MAR', 'Company Name': 'Marriott International', 'Start Price': 273.13, 'End Price': 273.47, 'Price Return %': 0.12, 'Q1 Dividends ($)': 1.3, 'Dividend Yield %': 0.48, 'Total Return %': 0.6, 'Volatility %': 34.06, 'Avg Daily Volume': 1798897, 'Momentum ($)': 0.34, 'R

$MTCH: possibly delisted; no price data found  (1d 1926-07-29 -> 2025-07-04)


[306/502] Checking: MKC
 Appending: {'Ticker': 'MKC', 'Company Name': 'McCormick & Company', 'Start Price': 75.43, 'End Price': 76.31, 'Price Return %': 1.16, 'Q1 Dividends ($)': 0.45, 'Dividend Yield %': 0.6, 'Total Return %': 1.76, 'Volatility %': 26.37, 'Avg Daily Volume': 2324934, 'Momentum ($)': 0.88, 'ROE': 14.12, 'P/E Ratio': 26.18}
[307/502] Checking: MCD
 Appending: {'Ticker': 'MCD', 'Company Name': "McDonald's", 'Start Price': 289.19, 'End Price': 291.55, 'Price Return %': 0.82, 'Q1 Dividends ($)': 3.54, 'Dividend Yield %': 1.22, 'Total Return %': 2.04, 'Volatility %': 21.15, 'Avg Daily Volume': 3602130, 'Momentum ($)': 2.36, 'ROE': None, 'P/E Ratio': 25.98}
[308/502] Checking: MCK
 Appending: {'Ticker': 'MCK', 'Company Name': 'McKesson Corporation', 'Start Price': 565.02, 'End Price': 725.78, 'Price Return %': 28.45, 'Q1 Dividends ($)': 1.42, 'Dividend Yield %': 0.25, 'Total Return %': 28.7, 'Volatility %': 21.84, 'Avg Daily Volume': 930846, 'Momentum ($)': 160.76, 'ROE': No

In [21]:
# Create a full dataframe of results
performers = pd.DataFrame(results)
print(performers)

    Ticker         Company Name  Start Price  End Price  Price Return %  \
0      MMM                   3M       128.43     152.02           18.36   
1      AOS          A. O. Smith        66.65      65.65           -1.50   
2      ABT  Abbott Laboratories       112.33     134.38           19.63   
3     ABBV               AbbVie       176.14     182.31            3.51   
4      ACN            Accenture       345.63     295.46          -14.52   
..     ...                  ...          ...        ...             ...   
493    XYL           Xylem Inc.       115.22     128.23           11.29   
494    YUM          Yum! Brands       132.28     148.41           12.19   
495   ZBRA   Zebra Technologies       383.76     309.26          -19.41   
496    ZBH        Zimmer Biomet       103.97      91.46          -12.03   
497    ZTS               Zoetis       161.57     156.17           -3.34   

     Q1 Dividends ($)  Dividend Yield %  Total Return %  Volatility %  \
0                1.46     

In [23]:
# Custom weighted scoring model to rank the best performing stocks

def normalize(series, inverse=False):
    norm = (series - series.min()) / (series.max() - series.min())
    return 1 - norm if inverse else norm

performers["Score_Value"] = normalize(performers["P/E Ratio"], inverse=True)
performers["Score_Growth"] = normalize(performers["Price Return %"])
performers["Score_Quality"] = normalize(performers["ROE"])
performers["Score_Yield"] = normalize(performers["Dividend Yield %"])
performers["Score_Stability"] = normalize(performers["Volatility %"], inverse=True)
performers["Score_Liquidity"] = normalize(performers["Avg Daily Volume"])


performers["Final Score"] = (
    0.25 * performers["Score_Quality"] +
    0.20 * performers["Score_Growth"] +
    0.15 * performers["Score_Value"]+
    0.15 * performers["Score_Yield"] +
    0.15 * performers["Score_Stability"] +
    0.10 * performers["Score_Liquidity"]
)

In [25]:
#  Rank the top 10 stocks based on final score
top_10_best = performers.nlargest(10, "Final Score") 
print(top_10_best[["Ticker", "Final Score"]])

    Ticker  Final Score
222    HCA     0.649194
197      F     0.575569
134    CVS     0.544283
299    MAS     0.524012
465     VZ     0.505620
47       T     0.503825
468   VICI     0.497689
388      O     0.493093
202    BEN     0.488294
189   FAST     0.483847


In [27]:
# Flag the final 10 with a True/False indicator
performers["Best Performers"] = performers["Ticker"].isin(top_10_best["Ticker"])

In [29]:
# Assign investment strategies dynamically using custom logic

def assign_strategy(row):
    
    if row["Ticker"] in aristocrat_tickers:
        return "Income-Focused"
    elif row["Price Return %"] >=10 and row["Dividend Yield %"] <1:
        return "Growth-Focused"
    elif row["Price Return %"] >=5 and row["Dividend Yield %"] >= 1:
        return "Balanced"
    else:
        return "Unclassified"

performers["Strategy"] = performers.apply(assign_strategy, axis=1)

In [31]:
# Select top 10 for each strategy using sorting metric
top_growth = performers[performers["Strategy"] == "Growth-Focused"].sort_values(by="Price Return %", ascending=False).head(10)
top_income = performers[performers["Strategy"] == "Income-Focused"].sort_values(by="Total Return %", ascending=False).head(10)
top_balanced = performers[performers["Strategy"] == "Balanced"].sort_values(by="Total Return %", ascending=False).head(10)
top_unclassified = performers[performers["Strategy"] == "Unclassified"].sort_values(by="Total Return %", ascending=False).head(10)

In [33]:
# Stack all top 10s across different strategies into one dataframe and flag the best performers based on final score
final_output = pd.concat([
    top_growth.assign(Strategy="Growth-Focused"),
    top_income.assign(Strategy="Income-Focused"),
    top_balanced.assign(Strategy="Balanced"),
    performers[performers["Best Performers"]]
]).drop_duplicates(subset=["Ticker", "Strategy"]).reset_index(drop=True)

print(final_output)

   Ticker                   Company Name  Start Price  End Price  \
0     NRG                     NRG Energy        91.98     162.67   
1    PLTR          Palantir Technologies        75.19     130.74   
2     HWM               Howmet Aerospace       110.63     184.90   
3    SMCI                     Supermicro        30.05      47.58   
4     GEV                     GE Vernova       338.68     519.66   
5     JBL                          Jabil       142.69     217.37   
6      GE                   GE Aerospace       168.28     254.51   
7    NFLX                        Netflix       886.73    1323.12   
8    UBER                           Uber        63.17      91.53   
9    CRWD                    CrowdStrike       347.34     499.33   
10    CAH                Cardinal Health       117.23     166.67   
11    IBM                            IBM       217.04     289.70   
12    BEN             Franklin Resources        19.54      23.82   
13   FAST                       Fastenal        

In [35]:
# Export to excel
final_output.to_excel("Top_10_Year_JuneEnd_2025.xlsx", index=False)
print("\n✅ Exported: Top_10_Year_JuneEnd_2025.xlsx")


✅ Exported: Top_10_Year_JuneEnd_2025.xlsx
